In [ ]:
!pip install transformers datasets accelerate torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from datasets import load_dataset

# MLSUM veri kümesini yükle
dataset = load_dataset("mlsum", "tu")
train_dataset = dataset["train"]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

mlsum.py:   0%|          | 0.00/3.72k [00:00<?, ?B/s]

The repository for mlsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mlsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/249277 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11565 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12775 [00:00<?, ? examples/s]

In [ ]:
# İlk 10,000 örneği seç
small_train_dataset = train_dataset.select(range(1000))


In [ ]:
small_validation_dataset = dataset["validation"].select(range(200))  # Örnek: 1000 validation verisi
small_test_dataset = dataset["test"].select(range(100))  # Örnek: 1000 test verisi


In [ ]:
from transformers import BigBirdPegasusForConditionalGeneration, AutoTokenizer

# BigBird tokenizer'ı yükle
model_name = "google/bigbird-pegasus-large-arxiv"
tokenizer = AutoTokenizer.from_pretrained("google/bigbird-pegasus-large-arxiv")

def split_text_with_sliding_window(text, max_token_length=1024, overlap=256):
    """
    Sliding Window yöntemiyle metni parçalara böler.
    :param text: Uzun metin (string)
    :param max_token_length: Modelin işleyebileceği maksimum token sayısı
    :param overlap: Çakışan token sayısı
    :return: Parçalara bölünmüş metinler listesi
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=False)
    input_ids = inputs["input_ids"][0]

    chunks = []
    for start in range(0, len(input_ids), max_token_length - overlap):
        end = min(start + max_token_length, len(input_ids))
        chunks.append(input_ids[start:end])
        if end == len(input_ids):  # Son parçaya ulaşıldığında döngüyü kır
            break

    return chunks

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.92M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.51M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

In [ ]:
# Uzun bir metni Sliding Window ile bölme
long_text = small_train_dataset[8]['text']
chunks = split_text_with_sliding_window(long_text)
print(f"Parça Sayısı: {len(chunks)}")


Parça Sayısı: 2


In [ ]:
from transformers import BigBirdPegasusForConditionalGeneration

# BigBird modelini yükle
model = BigBirdPegasusForConditionalGeneration.from_pretrained(model_name)


pytorch_model.bin:   0%|          | 0.00/2.31G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/232 [00:00<?, ?B/s]

In [ ]:
def preprocess_data(examples):
    inputs = examples['text']
    targets = examples['summary']

    # Tokenize işlemi
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Eğitim ve doğrulama setlerini işleme
tokenized_train = small_train_dataset.map(preprocess_data, batched=True)
tokenized_val = small_validation_dataset.map(preprocess_data, batched=True)
tokenized_test = small_test_dataset.map(preprocess_data, batched=True)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments, Trainer
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=1,  # GPU belleğine göre ayarlanabilir
    per_device_eval_batch_size=1,
    num_train_epochs=3,
    save_steps=1000,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=500,
    fp16=True,  # Eğer GPU destekliyorsa

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss
1,0.390200,0.578207
2,0.333500,0.543202
3,0.304900,0.529293


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 256, 'num_beams': 5, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3000, training_loss=0.45319554646809895, metrics={'train_runtime': 3284.1565, 'train_samples_per_second': 0.913, 'train_steps_per_second': 0.913, 'total_flos': 8664769363968000.0, 'train_loss': 0.45319554646809895, 'epoch': 3.0})

In [ ]:
import torch

# GPU kontrolü
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Modeli ve tokenizer'ı GPU'ya taşı
model = model.to(device)

# Tokenize edilmiş girdiyi GPU'ya taşı
def summarize_with_model(input_text):
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=4096)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Tüm tensorları GPU'ya taşı
    outputs = model.generate(**inputs, max_length=512, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Test
test_text = small_validation_dataset[4]['text']
predicted_summary = summarize_with_model(test_text)
print("Modelin Özetlediği:")
print(predicted_summary)


Modelin Özetlediği:
lk ekipleri yapt skan asker uurlamas srasnda midibs ile i makinas kafalar kafaya arpt.
